# Scraping GulAhmed - All Sections
## Retrieving images/details/names/sizes

In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
import urllib.request
import pandas as pd
import time
import requests
import json
import os

In [2]:
SectionNames = ['Unstitched', 'Ideas-Pret', 'Mens-Unstitched', 'Ideas-Home', 'Salt', 'Mens-Apparel']
sectionlinks = []

In [3]:
faultylinks = [[],[],[],[],[],[]]

In [4]:
base_link = 'https://www.gulahmedshop.com/'
uClient = uReq(base_link)
all_web = uClient.read()
uClient.close()
page_soup = soup(all_web, "html.parser")
sections = page_soup.find_all("div", {'class':'groupbanner-text hover-scale'})
for section in sections:
    sectionlinks.append(base_link + section.a['href'])

In [5]:
pages = [[],[],[],[],[],[]]
productlinks = [[],[],[],[],[],[]]
all_rows = [[],[],[],[],[],[]]

In [6]:
%%time
for i in range(len(sectionlinks)):
    uClient = uReq(sectionlinks[i])
    all_web = uClient.read()
    uClient.close()
    page_soup = soup(all_web, "html.parser")
    pages_soup = page_soup.find('div', {'class':'pages'})('li')
    pages[i].append(sectionlinks[i])
    for li in pages_soup:
        if (li['class'] != ['item', 'current'] and li['class'] != ['item', 'pages-item-next']):
            pages[i].append(li.a['href'])
            if (li.a['href'][-1] == '5'):
                uClient = uReq(li.a['href'])
                all_web = uClient.read()
                uClient.close()
                page_soup = soup(all_web, "html.parser")
                pages2_soup = page_soup.find('div', {'class':'pages'})('li')
                flag = 0
                for lii in pages2_soup:
                    #print(lii)
                    if (flag == 1 and lii['class'] != ['item', 'pages-item-next']):
                        pages[i].append(lii.a['href'])
                    if (lii['class'] == ['item', 'current']):
                        flag = 1
print('completed')

completed
Wall time: 1min 4s


In [7]:
%%time
h = -1
for i in pages:
    h += 1
    for i2 in i:
        uClient = uReq(i2)
        all_web = uClient.read()
        uClient.close()
        page_soup = soup(all_web, "html.parser")
        v = page_soup.findAll('a', {'class': 'product photo product-item-photo'})
        for link in v:
            productlinks[h].append(link['href'])
print("completed")

completed
Wall time: 5min 44s


In [8]:
for z in range(6):
    print(len(productlinks[z]))

165
315
99
315
84
315


In [9]:
with open('listfile.txt', 'w') as filehandle:
    json.dump(productlinks, filehandle)
    
#with open('listfile.txt', 'r') as filehandle:
    #productlinks = json.load(filehandle)

In [9]:
print(len(productlinks[3]))
#print(len(productlinkstest[2]))


0


In [10]:
def find_GUL_size(link):
    #link=requests.get(link).text
    parse=soup(link,"html.parser")
    size_str=""
    data=parse.find("div",class_="product-options-wrapper")
    if(data is not None):
        data=parse.text
        #print(data)
        start=str(data).find("\"options\":")
        #print(start)
        end=str(data).find("}],\"position\":\"0\"}}")
        #print(end)
        json_extract=(str(data)[start:end]+"}]").replace("\"options\":","")
        #print(json_extract)
        loaded=json.loads(json_extract)
        for sizes in loaded:
            temp=sizes["products"]
            if(str(temp).find("[]") is -1):
                size_str+=sizes["label"]+"/"
        size_str=size_str[0:(len(size_str)-1)]
    return size_str

In [12]:
%%time

for i in range(len(sectionlinks)):
    path = 'G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Gul-Ahmed/' + SectionNames[i]
    if not os.path.exists(path):
        os.mkdir(path)
    h = 0
    try:    
        for link in productlinks[i]:
            h+= 1
            if (h%10 == 0):
                print('In Process: ' + str(h))
            uClient = uReq(link)
            all_web = uClient.read()
            uClient.close()
            name = None
            newprice = None
            oldprice = None
            sizes = None
            description = None
            page_soup = soup(all_web, "html.parser")
            name = page_soup.find('h1', {'class':'page-title'}).text.strip()
            prices = page_soup.findAll('span', {'class': 'price'})
            newprice = prices[0].text.strip()
            oldprice = prices[1].text.strip()
            discount = round(((int(oldprice.replace("PKR", "").replace(",", "")) - int(newprice.replace("PKR", "").replace(",", "")))/int(oldprice.replace("PKR", "").replace(",", ""))) * 100)
            sizes = find_GUL_size(all_web)
            s = page_soup.find('div', class_="product attribute description")
            if (s is not None):
                s = s.text
            try:
                n = s.find('Reasons To Shop From Ideas')
                description = s[:n].strip()
            except:
                if (s is not None):
                    description = s.strip()
                pass
            
            #images:
            #i = page_soup.find("div", {'class':'MagicToolboxSelectorsContainer'})('a')
            #m = -1  
            #for a in i:
            #    m += 1
            #    filename = str(h) + '.' + str(m) + '.jpg'
            #    urllib.request.urlretrieve(a['href'], os.path.join(path, filename))
                
                
            row = {'Name':name, 'Description': description, 'Old Price': oldprice, 'New Price': newprice,'Discount' : discount, 'Size(s)': sizes}
            all_rows[i].append(row)
    except:
        print(link)
        faultylinks[i].append(link)
        pass

In Process: 10
In Process: 20
In Process: 30
In Process: 40
In Process: 50
In Process: 60
In Process: 70
In Process: 80
In Process: 90
In Process: 100
In Process: 110
In Process: 120
In Process: 130
In Process: 140
In Process: 150
In Process: 160
In Process: 170
In Process: 180
In Process: 190
In Process: 200
In Process: 210
In Process: 220
In Process: 230
https://www.gulahmedshop.com/cambric-kurti-gls-19-296-dp
In Process: 240
In Process: 250
In Process: 260
In Process: 270
In Process: 280
In Process: 290
In Process: 300
https://www.gulahmedshop.com/brown-opus-paradise-unstitched-fabric
In Process: 310
In Process: 320
In Process: 330
In Process: 340
In Process: 350
In Process: 360
In Process: 370
In Process: 380
In Process: 390
In Process: 400
In Process: 410
In Process: 420
In Process: 430
In Process: 440
In Process: 450
In Process: 460
In Process: 470
In Process: 480
In Process: 490
In Process: 500
In Process: 510
In Process: 520
In Process: 530
In Process: 540
In Process: 550
In Pr

In [14]:
products = pd.DataFrame(all_rows[1], columns = ['Name','Description','Old Price','New Price','Discount', 'Size(s)'])
products


,Name,Description,Old Price,New Price,Discount,Size(s)
0,Khaddar Shirt GLW-19-15,"Maroon, screen-printed dyed khaddar shirt with...","PKR 3,790","PKR 1,895",50,X-Large
1,3 PC Cambric Outfit IPW-19-72,"Multi-colored, digital printed cambric kurti w...","PKR 4,950","PKR 2,970",40,X-Small/Small/Medium/X-Large
2,3 PC Cambric Outfit IPW-19-71,"Multi-colored, digital printed cambric kurti w...","PKR 4,950","PKR 2,970",40,Small/Medium/X-Large
3,3 PC Cambric Outfit IPW-19-70,"Multi-colored, digital printed cambric kurti w...","PKR 4,950","PKR 2,970",40,X-Small/Small/X-Large
4,3 PC Cambric Outfit IPW-19-69,"Multi-colored, digital printed cambric kurti w...","PKR 4,950","PKR 2,970",40,X-Small/Medium/X-Large
5,3 PC Cambric Outfit IPW-19-68,"Multi-colored, digital printed cambric kurti w...","PKR 4,950","PKR 2,970",40,X-Small/Small/Medium/Large
6,3 PC Cambric Outfit IPW-19-67,"Multi-colored, digital printed cambric kurti w...","PKR 4,950","PKR 2,970",40,X-Small/Small/X-Large
7,3 PC Cambric Outfit IPW-19-66,"Multi-colored, digital printed cambric kurti w...","PKR 4,950","PKR 2,970",40,X-Small/Small/Medium/Large/X-Large
8,3 PC Cambric Outfit IPW-19-65,"Multi-colored, digital printed cambric kurti w...","PKR 4,950","PKR 2,970",40,X-Small/Small/Medium/X-Large
9,3 PC Cambric Outfit IPW-19-63,"Multi-colored, digital printed cambric kurti w...","PKR 4,950","PKR 2,970",40,X-Small/Small/Medium/Large/X-Large


In [24]:
link = 'https://www.gulahmedshop.com/wse-73'
h = 0
uClient = uReq(link)
all_web = uClient.read()
uClient.close()
name = None
newprice = None
oldprice = None
sizes = None
description = None
page_soup = soup(all_web, "html.parser")
name = page_soup.find('h1', {'class':'page-title'}).text.strip()
prices = page_soup.findAll('span', {'class': 'price'})
newprice = prices[0].text.strip()
oldprice = prices[1].text.strip()
sizes = find_GUL_size(all_web)
description = None
s = page_soup.find('div', class_="product attribute description")
discount = round(((int(oldprice.replace("PKR", "").replace(",", "")) - int(newprice.replace("PKR", "").replace(",", "")))/int(oldprice.replace("PKR", "").replace(",", ""))) * 100)
if s is not None:
    s = s.text
try:
    n = s.find('Reasons To Shop From Ideas')
    description = s[:n].strip()
except:
    if (s is not None):
        description = s.strip()
    pass
#images:

    
print(discount, newprice)

30 PKR 3,190


In [28]:
%%time
for i in range(len(sectionlinks)):
    products = pd.DataFrame(all_rows[i], columns = ['Name','Description','Old Price','New Price','Discount', 'Size(s)'])
    path = 'G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Gul-Ahmed/' + SectionNames[i]
    print(path)
    products.to_csv(os.path.join(path, SectionNames[i] + "1.csv"), encoding='utf-8')

G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Gul-Ahmed/Unstitched
G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Gul-Ahmed/Ideas-Pret
G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Gul-Ahmed/Mens-Unstitched
G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Gul-Ahmed/Ideas-Home
G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Gul-Ahmed/Salt
G:/My Drive/FAST!!/1st Semester/Co-Curricular/DSC - Outreach/Saad/ScrapIt/Gul-Ahmed/Mens-Apparel
Wall time: 98.7 ms
